In [1]:
import sys
import os
import mxnet as mx
import mxnet.metric #import EvalMetric
import numpy as np
import logging
# code to automatically download dataset
mxnet_path = os.path.dirname(os.path.abspath(os.path.expanduser(mxnet.__file__)))
sys.path.append(os.path.join(mxnet_path, "../../tests/python/common"))
sys.path.append(os.path.join(mxnet_path, "../../example/python-howto"))
from data import mnist_iterator
import get_data

In [2]:
def perDigitMetric(params):
    #print "Metric", params
    for i, cm in enumerate(params.eval_metric.cms):
        print "Label=", i
        print cm

class MNISTPerDigitAccuracy(mx.metric.EvalMetric):
    """Calculate accuracy"""

    def reset(self):
        if hasattr(self, 'cms'):
            for cm in self.cms:
                cm.fill(0)
        

    def __init__(self,sizes):
        super(MNISTPerDigitAccuracy, self).__init__('mnistperdigiaccuracy')
        self.cms = map( lambda x: np.zeros((x,x), dtype=int), sizes )
        self.reset()
        return super(MNISTPerDigitAccuracy, self).reset()


    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        #print labels, preds
        for i in range(len(labels)):
            for label, pred_label in zip(labels[i].asnumpy(), preds[i].asnumpy()):
                #pred_label = int(np.argmax(pred_label))
                #label = int(label)
                #print "LABEL=", label, "PRED=", pred_label
                l = np.argmax(label[0:10])
                p = np.argmax(pred_label[0:10])
                self.cms[i][l,p] += 1
                self.sum_metric += (l == p)
                
                l = 10 + np.argmax(label[10:12])
                p = 10 + np.argmax(pred_label[10:12])
                self.cms[i][l,p] += 1
                self.sum_metric += (l == p)

                self.num_inst += 2


In [3]:
logging.basicConfig(level=logging.DEBUG)

In [4]:
train, val = mnist_iterator(batch_size=100, input_shape = (784,))

In [5]:
class EvenOdd_iterator(mx.io.DataIter):
    '''multi label ilab iterator'''

    def __init__(self, data_iter, parentweight):
        super(EvenOdd_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size
        self.parentweight = parentweight
        
    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        #return [mx.io.DataDesc('softmax_label', (self.batch_size,), np.float32)]
        return [mx.io.DataDesc('softmax_label', (self.batch_size,12), np.float32)]

    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        #labels= batch.label
        labels = []
        if True:
            digitlabels = np.zeros((self.batch_size,12))
            digitlabels[np.arange(self.batch_size),batch.label[0].asnumpy().astype(int)]=1-self.parentweight
            eolabels = 10 + (batch.label[0].asnumpy().astype(int) % 2 )
            digitlabels[np.arange(self.batch_size),eolabels]=self.parentweight
            digitlabels = mx.nd.array(digitlabels)
            #print batch.label[0].asnumpy().astype(int)
            #print digitlabels.asnumpy()
            #fdafdaf
        else:
            digitlabels = mx.nd.zeros((12,))
        #print digitlabels
        #print batch.label[0].asnumpy()
        labels.append(digitlabels)
        #print labels
        #eolabels = []
        #for i in batch.label[0].asnumpy():
        #    eolabels.append(i%2)
        #eolabels = mx.nd.array(np.array(eolabels))
        #labels.append(eolabels)
        return mx.io.DataBatch(data=batch.data, label=labels, pad=batch.pad, index=batch.index)


In [6]:
parentweight = 0.25
_train = EvenOdd_iterator(train,parentweight)
_val = EvenOdd_iterator(train,parentweight)

In [7]:
data = mx.symbol.Variable('data')
fc1 = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2 = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3 = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=12)

In [8]:
mlp = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')
optimizer_params = (('learning_rate', 0.1),('momentum', 0.9), ('wd', 0.00001))

In [9]:
mod = mx.mod.Module(mlp,label_names=['softmax_label'])
mod.label_names
mod.fit(_train, 
        eval_data=_val,
        optimizer_params=optimizer_params,
        eval_metric=MNISTPerDigitAccuracy([12]),
        eval_end_callback=perDigitMetric,
        num_epoch=10, batch_end_callback=mx.callback.Speedometer(100,100))

INFO:root:Epoch[0] Batch [100]	Speed: 35902.76 samples/sec	Train-mnistperdigiaccuracy=0.305891
INFO:root:Epoch[0] Batch [200]	Speed: 38614.37 samples/sec	Train-mnistperdigiaccuracy=0.465721
INFO:root:Epoch[0] Batch [300]	Speed: 32149.47 samples/sec	Train-mnistperdigiaccuracy=0.609336
INFO:root:Epoch[0] Batch [400]	Speed: 35740.95 samples/sec	Train-mnistperdigiaccuracy=0.690012
INFO:root:Epoch[0] Batch [500]	Speed: 35281.09 samples/sec	Train-mnistperdigiaccuracy=0.741747
INFO:root:Epoch[0] Train-mnistperdigiaccuracy=0.776658
INFO:root:Epoch[0] Time cost=1.708
INFO:root:Epoch[0] Validation-mnistperdigiaccuracy=0.870642


Label= 0
[[ 5780     0     7     4     7    24    62    11    28     0     0     0]
 [    1  6612    31    17     7     2     8    21    35     8     0     0]
 [   24    18  5671    56    32     2    62    36    52     5     0     0]
 [    4    27    63  5847     3    70     5    35    59    18     0     0]
 [    6    15    11     3  5500     3   122     8     8   166     0     0]
 [   32     9     9   157    25  5021    63    17    53    35     0     0]
 [   17     7     6     1    12    40  5818     0    17     0     0     0]
 [    7    27    41    19    36     4     3  6077     2    49     0     0]
 [   12    54    21    77    35    47    60     8  5486    51     0     0]
 [   31    10     4    60   164    28     3   192    29  5428     0     0]
 [    0     0     0     0     0     0     0     0     0     0 28780   712]
 [    0     0     0     0     0     0     0     0     0     0   773 29735]]


INFO:root:Epoch[1] Batch [100]	Speed: 25727.52 samples/sec	Train-mnistperdigiaccuracy=0.877467
INFO:root:Epoch[1] Batch [200]	Speed: 24720.41 samples/sec	Train-mnistperdigiaccuracy=0.883829
INFO:root:Epoch[1] Batch [300]	Speed: 28992.64 samples/sec	Train-mnistperdigiaccuracy=0.889420
INFO:root:Epoch[1] Batch [400]	Speed: 32639.20 samples/sec	Train-mnistperdigiaccuracy=0.894391
INFO:root:Epoch[1] Batch [500]	Speed: 27180.99 samples/sec	Train-mnistperdigiaccuracy=0.898948
INFO:root:Epoch[1] Train-mnistperdigiaccuracy=0.903050
INFO:root:Epoch[1] Time cost=2.122
INFO:root:Epoch[1] Validation-mnistperdigiaccuracy=0.922233


Label= 0
[[ 5842     0     6     3     5     4    38     3    19     3     0     0]
 [    1  6566    21    30     8     0    12    14    70    20     0     0]
 [   20     6  5798    29    17     1    20    18    44     5     0     0]
 [    5     1    55  5946     0    43     3    14    46    18     0     0]
 [    6     0    13     0  5660     0    64     5     9    85     0     0]
 [   18     1     5    54     5  5267    31     4    21    15     0     0]
 [   17     2     5     0     8    21  5854     0    11     0     0     0]
 [    7    13    57    32    16     4     1  6038    11    86     0     0]
 [   12     6    12    24     7    19    21     2  5738    10     0     0]
 [   24     2     1    29    69    23     1    31    38  5731     0     0]
 [    0     0     0     0     0     0     0     0     0     0 29213   279]
 [    0     0     0     0     0     0     0     0     0     0   587 29921]]


INFO:root:Epoch[2] Batch [100]	Speed: 35134.29 samples/sec	Train-mnistperdigiaccuracy=0.924336
INFO:root:Epoch[2] Batch [200]	Speed: 35301.25 samples/sec	Train-mnistperdigiaccuracy=0.926459
INFO:root:Epoch[2] Batch [300]	Speed: 32160.96 samples/sec	Train-mnistperdigiaccuracy=0.928365
INFO:root:Epoch[2] Batch [400]	Speed: 28055.45 samples/sec	Train-mnistperdigiaccuracy=0.930214
INFO:root:Epoch[2] Batch [500]	Speed: 25058.63 samples/sec	Train-mnistperdigiaccuracy=0.931908
INFO:root:Epoch[2] Train-mnistperdigiaccuracy=0.933528
INFO:root:Epoch[2] Time cost=2.010
INFO:root:Epoch[2] Validation-mnistperdigiaccuracy=0.941769


...

Label= 0
[[ 5896     0     5     2     1     0     9     0     7     3     0     0]
 [    0  6669     7     6     6     0     5    15    33     1     0     0]
 [    8     1  5927     7     3     0     0     6     5     1     0     0]
 [    1     2    28  6056     0     6     0     3    31     4     0     0]
 [    0     1     1     0  5825     0     3     1     4     7     0     0]
 [    7     1     1    14     0  5350    25     0    21     2     0     0]
 [    9     0     3     0    10     1  5880     0    15     0     0     0]
 [    1     2    24    19     1     2     0  6197     9    10     0     0]
 [    1     0    12     3     1     2     1     1  5830     0     0     0]
 [    3     0     1    38    47    13     0    20    37  5790     0     0]
 [    0     0     0     0     0     0     0     0     0     0 29458    34]
 [    0     0     0     0     0     0     0     0     0     0   367 30141]]


INFO:root:Epoch[8] Batch [100]	Speed: 25804.99 samples/sec	Train-mnistperdigiaccuracy=0.971796
INFO:root:Epoch[8] Batch [200]	Speed: 29115.04 samples/sec	Train-mnistperdigiaccuracy=0.972028
INFO:root:Epoch[8] Batch [300]	Speed: 29994.54 samples/sec	Train-mnistperdigiaccuracy=0.972243
INFO:root:Epoch[8] Batch [400]	Speed: 28522.71 samples/sec	Train-mnistperdigiaccuracy=0.972457
INFO:root:Epoch[8] Batch [500]	Speed: 33096.59 samples/sec	Train-mnistperdigiaccuracy=0.972663
INFO:root:Epoch[8] Train-mnistperdigiaccuracy=0.972867
INFO:root:Epoch[8] Time cost=2.047
INFO:root:Epoch[8] Validation-mnistperdigiaccuracy=0.974017


Label= 0
[[ 5908     0     2     0     1     0     9     0     2     1     0     0]
 [    0  6702     6     1     1     0     5    14    12     1     0     0]
 [    7     3  5940     2     2     0     0     1     3     0     0     0]
 [    2     5    37  6044     0     7     0     6    23     7     0     0]
 [    0     1     0     0  5831     0     4     1     1     4     0     0]
 [    8     1     1    14     5  5350    29     0     8     5     0     0]
 [    4     0     4     0    14     1  5888     0     7     0     0     0]
 [    0     3    16     2     3     0     0  6237     0     4     0     0]
 [    5     2    10     0     7     0     6     1  5820     0     0     0]
 [    8     4     0    14    57     4     0    19    12  5831     0     0]
 [    0     0     0     0     0     0     0     0     0     0 29479    13]
 [    0     0     0     0     0     0     0     0     0     0   310 30198]]


INFO:root:Epoch[9] Batch [100]	Speed: 31315.48 samples/sec	Train-mnistperdigiaccuracy=0.974204
INFO:root:Epoch[9] Batch [200]	Speed: 26824.11 samples/sec	Train-mnistperdigiaccuracy=0.974399
INFO:root:Epoch[9] Batch [300]	Speed: 24537.54 samples/sec	Train-mnistperdigiaccuracy=0.974578
INFO:root:Epoch[9] Batch [400]	Speed: 35816.36 samples/sec	Train-mnistperdigiaccuracy=0.974771
INFO:root:Epoch[9] Batch [500]	Speed: 36022.52 samples/sec	Train-mnistperdigiaccuracy=0.974953
INFO:root:Epoch[9] Train-mnistperdigiaccuracy=0.975124
INFO:root:Epoch[9] Time cost=1.951
INFO:root:Epoch[9] Validation-mnistperdigiaccuracy=0.975789


Label= 0
[[ 5895     0     6     0     0     0    16     0     6     0     0     0]
 [    0  6657    22     0     6     0     3    13    38     3     0     0]
 [    5     1  5943     0     2     0     0     1     6     0     0     0]
 [    1     5   106  5931     0     2     0     8    72     6     0     0]
 [    0     0     0     0  5834     0     3     1     3     1     0     0]
 [   13     1     1    14     3  5291    40     0    52     6     0     0]
 [    2     1     8     0     6     0  5891     0    10     0     0     0]
 [    0     1    42     0     4     0     0  6204     4    10     0     0]
 [    2     0    12     0     1     0     0     0  5836     0     0     0]
 [    7     0     1     7    84     3     0    16    33  5798     0     0]
 [    0     0     0     0     0     0     0     0     0     0 29488     4]
 [    0     0     0     0     0     0     0     0     0     0   665 29843]]
